<a href="https://colab.research.google.com/github/MohammedHamood/20NewsGroup/blob/main/20NewsGroup_AdaBoost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin

import time
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

import pipelines_FEngineering as BaseLine
from sklearn.datasets import fetch_20newsgroups
import preprocessingNLP as PNLP

from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA, NMF,TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import Normalizer

# Import Dataset
TNGD_Train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42,remove=['headers', 'footers', 'quotes'])
TNGD_test = fetch_20newsgroups(subset='test',shuffle=True, random_state=42,remove=['headers', 'footers', 'quotes'])

# Preprocessing
print("PREPROCESSING ...")
TNGD_Train.data = PNLP.customNLP(TNGD_Train.data)
TNGD_test.data = PNLP.customNLP(TNGD_test.data)
TNGD_Train.data, TNGD_Train.target = PNLP.removeEmptyInstances(TNGD_Train.data, TNGD_Train.target)



print("PREPROCESSING DONE!")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


PREPROCESSING ...


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


PREPROCESSING DONE!


In [ ]:

# Utility function to report best scores
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("Mean Fit Time: %.3f seconds" %results['mean_fit_time'][candidate])
            print("")


from scipy import sparse
def Pipeline_FeatureEngineering(train_data,train_target,parameters=None,CV=10, 
                                reductionMethod=PCA(),reductionType=1,model=LinearSVC()):
    class DenseTransformer(TransformerMixin):

        def fit(self, X, y=None, **fit_params):
            return self

        def transform(self, X, y=None, **fit_params):
            return X.todense()
    class sparseTransformer(TransformerMixin):

        def fit(self, X, y=None, **fit_params):
            return self

        def transform(self, X, y=None, **fit_params):
            return sparse.csr_matrix(X) 
        
 

    class dcdistanceTransformer(TransformerMixin, BaseEstimator): 
        """
        proposed method modifed from(DCDistance: A Supervised Text Document Feature extraction based on class labels)
        """
        def __init__(self):
            super(dcdistanceTransformer, self).__init__()
            

        def fit(self, X, y, **fit_params):
            self.alpha=20
            self.Classes=np.unique(y, return_index = False)
            self.split=np.linspace(0, X.shape[1],self.alpha,endpoint = False,  dtype=int)
            self.vd=np.zeros((len(self.Classes),X.shape[1]))
            self.vd2=np.zeros((len(self.Classes),X.shape[1]))
            for i in self.Classes:
                ind = np.where(y == i)[0]
                temp=np.zeros((1,X.shape[1]))
                for feat in range(0,len(self.split)):
                    ind1=self.split[feat]              
                    ind2=self.split[feat]+self.split[1]
                    temp=X.tocsr()[ind,ind1:ind2].sum(axis = 0)
                    self.vd2[int(i),ind1:ind2]=temp     
            tfidf_transformer = TfidfTransformer()
            self.vd2 = tfidf_transformer.fit_transform(self.vd2) 
            self.vd2=self.vd2.todense()   
            return self

        def transform(self, X, **fit_params):
            V= np.zeros((X.shape[0],len(self.Classes)))

            
            for i in self.Classes:
                starting=0
                ending=200
                while starting<X.shape[0]-2:
                    if ending>X.shape[0]:
                        ending=X.shape[0]
                    temp=np.linalg.norm(X.tocsr()[starting:ending,:]-self.vd2[int(i),:],axis=1,ord=2)
                    V[starting:ending,int(i)]= temp[:] #('Norm',Normalizer(copy=False)),
                    starting=ending
                    ending=ending+200
            V= sparse.csr_matrix(V)
            X=[]
            return V         
#
        
    if reductionType==1:
        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2),max_features =120000)),('tfidf', TfidfTransformer()),
                             ('redu',dcdistanceTransformer()),('clf', model)])
    elif reductionType==2:
        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),('tfidf', TfidfTransformer()),
                             ('redu',reductionMethod),
                             ('Norm',Normalizer(copy=False)),('clf', model)])
    elif reductionType==3:
        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),('tfidf', TfidfTransformer()),
                             ('Norm1',Normalizer(copy=False)),('clf', model)])    
    elif reductionType==0:
        text_clf = Pipeline([('vect', CountVectorizer()),('samp',ADASYN(random_state=42)),('tfidf', TfidfTransformer()),
                             ('redu1',TruncatedSVD(n_components=300)),('Norm1',Normalizer(copy=False)),('redu',reductionMethod),
                             ('Norm2',Normalizer(copy=False)),('clf', model)])
    gs_clf = GridSearchCV(text_clf, parameters, cv=CV)
    gs_clf = gs_clf.fit(train_data, train_target)
    return gs_clf



In [ ]:

parameters = {
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 1000,5000, 10000, 50000),
    #'vect__ngram_range': ((1, 1), (1, 2),(1, 3)), 
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__solver': ('lbfgs'),
    'clf__n_estimators': (100,80),
    #'clf__base_estimator':[RandomForestClassifier()],
    #'clf__base_estimator__criterion': ('gini','entropy'),
    #'clf__base_estimator__ccp_alpha': (0,.4),
    #'clf__base_estimator__max_depth': (None,5,20,100),
    #'clf__base_estimator__min_samples_split': (2,10,50,100),
    #'clf__base_estimator__min_samples_leaf': (1,10,50,100),
    #'clf__base_estimator__max_features': ('auto', 'log2'),
    #'clf__base_estimator__max_leaf_nodes': (None,20,30,40,80),
    }

# Initialize gridsearchcv
cv_folds=11
ada_RF_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=3,model=AdaBoostClassifier())

# Execute gridsearchcv and print best results 

report(ada_RF_base.cv_results_)


test_pridect=ada_RF_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))



Model with rank: 1
Mean validation score: 0.483 (std: 0.015)
Parameters: {'clf__n_estimators': 100}
Mean Fit Time: 226.083 seconds

Model with rank: 2
Mean validation score: 0.472 (std: 0.016)
Parameters: {'clf__n_estimators': 80}
Mean Fit Time: 164.433 seconds

              precision    recall  f1-score   support

           0       0.32      0.47      0.38       220
           1       0.43      0.55      0.49       307
           2       0.36      0.53      0.43       268
           3       0.44      0.49      0.46       358
           4       0.43      0.70      0.54       237
           5       0.43      0.57      0.49       296
           6       0.57      0.60      0.58       372
           7       0.39      0.79      0.52       195
           8       0.48      0.89      0.62       213
           9       0.48      0.77      0.59       246
          10       0.63      0.71      0.67       357
          11       0.51      0.77      0.62       262
          12       0.64      0.10 

In [ ]:



"""
 AdaBoostClassifier train with SVM 
"""
parameters = {
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 1000,5000, 10000, 50000),
    #'vect__ngram_range': ((1, 1), (1, 2),(1, 3)), 
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__solver': ('lbfgs'),
    'clf__base_estimator':[LinearSVC()],
    'clf__n_estimators': (8,20,50),
    #'clf__n_estimators__tol': (.001,.0001), 
    #'clf__base_estimator__C': (1000,10, 5,1,.5),
    }


"""
 AdaBoostClassifier  test with SVM
"""
# Initialize gridsearchcv
cv_folds=10
ada_svm_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=3,model=AdaBoostClassifier())

# Execute gridsearchcv and print best results 

report(ada_RF_base.cv_results_)


test_pridect=ada_svm_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))



In [ ]:


"""
 AdaBoostClassifier with LR  
"""
parameters = {
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 1000,5000, 10000, 50000),
    #'vect__ngram_range': ((1, 1), (1, 2),(1, 3)), 
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__solver': ('lbfgs'),
    #'clf__base_estimator':[LogisticRegression()],
    'clf__n_estimators': (50,80,100),
    #'clf__base_estimator__tol': (.001,.0001),
    'clf__base_estimator__C': (10, 5, 1),
    }

# Initialize gridsearchcv
cv_folds=11
ada_lr_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=3,model=AdaBoostClassifier(base_estimator=LogisticRegression()))

# Execute gridsearchcv and print best results 

report(ada_lr_base.cv_results_)


test_pridect=ada_RF_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))

KeyboardInterrupt: ignored